In [226]:
import pandas as pd 
import numpy as np
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from stop_words import get_stop_words
from nltk.tokenize import word_tokenize

#puede que ya no se necesiten estos paquetes
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /home/ernesto_anaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [227]:
#mongoexport --host localhost --db apilegislativo --collection juntos_harem_hist --type=csv --out alianza_morena.csv --fields _id,partidos

#iniciativas va por méxico
va_x_mex = pd.read_csv("inics_va_x_mex.csv")
#iniciativas de juntos haremos historia
alianza_morena = pd.read_csv("alianza_morena.csv")

iniciativas = pd.read_csv("inics_aprobs.csv")

In [284]:
def remove_stopwords(text):
    
    tokens = word_tokenize(text)
    custom_stopwords = stop_words_sp + ['accesos', 'sugestión']
    text_wo_stopwords = [word for word in tokens if not word in custom_stopwords]

    return ' '.join(text_wo_stopwords)

In [229]:
iniciativas

,id,title,abstract,authors,laws_mod,status_date
0,5489,proyecto de decreto que reforma el artículo 25...,tipificar los delitos en materia de hostigamie...,alonzo morales maria ester,código penal federal,"Tue, 18 Feb 2020 00:00:00 GMT"
1,5950,proyecto de decreto para inscribir en letras d...,inscribir en letras de oro en el muro de honor...,muñoz ledo porfirio,NaN,"Thu, 27 Jun 2019 00:00:00 GMT"
2,6132,proyecto de decreto por el que se reforman div...,establecer que en todos los niveles educativos...,martinez ruiz maribel,ley general de educación,"Thu, 19 Sep 2019 00:00:00 GMT"
3,6114,proyecto de decreto por el que se reforman y a...,establecer mecanismos para erradicar la violen...,cruz santos flora tania,ley general de educación,"Thu, 19 Sep 2019 00:00:00 GMT"
4,6584,proyecto de decreto que adiciona un artículo 4...,tipificar como delito los actos de crueldad y ...,herrera chavez samuel,código penal federal,"Tue, 23 Mar 2021 00:00:00 GMT"
...,...,...,...,...,...,...
1557,12179,proyecto de decreto por el que reforma el artí...,dejar sin efectos la facultad de la comisión r...,rodriguez gonzalez manuel,ley de hidrocarburos,"Wed, 21 Apr 2021 00:00:00 GMT"
1558,12206,proyecto de decreto por el que se adiciona un ...,establecer el resguardo de iniciativas y minut...,grupos parlamentarios,NaN,"Tue, 20 Apr 2021 00:00:00 GMT"
1559,12227,"proyecto de decreto por el que se reforman, ad...","incluir las definiciones de avión (aeroplano),...",ejecutivo federal,ley de aviación civil,"Wed, 28 Apr 2021 00:00:00 GMT"
1560,12245,proyecto de decreto que adiciona un artículo 3...,establecer como delito de fraude el publicitar...,romo cuellar martha estela,código penal federal,"Wed, 28 Apr 2021 00:00:00 GMT"


In [230]:
va_x_mex.rename(columns={col: col.lower() for col in iniciativas.columns.values}, inplace=True)
alianza_morena.rename(columns={col: col.lower() for col in iniciativas.columns.values}, inplace=True)

In [231]:
#limpieza inicial de las tablas

#de la tabla con las iniciativas de va por méxico
va_x_mex[['id','status','abstract','turno','laws_mod','status_date','state_pres','sess_pres']] = va_x_mex['_id'].str.split('","',expand=True)
va_x_mex[['id','title']] = va_x_mex['id'].str.split(',"',expand=True)
va_x_mex['id'] = va_x_mex['id'].str.split('":').str[-1].str.strip()
del va_x_mex['_id']

columns = ['id','status', 'abstract','turno','laws_mod','status_date','state_pres','sess_pres','title']
for column in columns:
    va_x_mex[column] = va_x_mex[column].str.split('":"').str[-1].str.strip()

#de la tabla con las iniciativas de juntos haremos historia
alianza_morena[['id','status','abstract','turno','laws_mod','status_date','state_pres','sess_pres']] = alianza_morena['_id'].str.split('","',expand=True)
alianza_morena[['id','title']] = alianza_morena['id'].str.split(',"',expand=True)
alianza_morena['id'] = alianza_morena['id'].str.split('":').str[-1].str.strip()

del alianza_morena['_id']

columns = ['id','status', 'abstract','turno','laws_mod','status_date','state_pres','sess_pres','title']
for column in columns:
    alianza_morena[column] = alianza_morena[column].str.split('":"').str[-1].str.strip()


va_x_mex

,partidos,id,status,abstract,turno,laws_mod,status_date,state_pres,sess_pres,title
0,"[""morena""]",11470,Aprobada,incluir a los hombres y a las personas adultas...,Salud,ley de asistencia social,"Wed, 21 Apr 2021 00:00:00 GMT",ciudad de mexico,versión estenográfica de la sesión ordinaria d...,proyecto de decreto que adiciona diversas disp...
1,"[""morena""]",5949,retirada,sustituir la comisión de apelación y arbitraje...,Deporte,ley general de cultura física y deporte,"Tue, 11 Jun 2019 00:00:00 GMT",ciudad de mexico,versión estenográfica de la sesión de congreso...,proyecto de decreto que deroga y adiciona dive...
2,"[""morena""]",3579,Pendiente,propone establecer que las personas adultas ma...,"Puntos Constitucionales , Estudios Legislativo...",Constitución Política de los Estados Unidos Me...,ciudad de mexico,sesión ordinaria a distancia de la h. cámara ...,None,proyecto de decreto por el que se reforman los...
3,"[""morena""]",3422,Pendiente,propone que cuando el ejecutivo federal emita ...,"Seguridad Social , Estudios Legislativos Segunda",Ley del Seguro Social,ciudad de mexico,sesión ordinaria a distancia de la h. cámara ...,None,proyecto de decreto por el que se adicionan lo...
4,"[""morena""]",587,Pendiente,propone integrar en el texto de la ley general...,"Salud , Estudios Legislativos Segunda",Ley General de Salud,ciudad de mexico,sesión ordinaria de la h. cámara de senadores...,None,proyecto de decreto que reforma el artículo 74...
...,...,...,...,...,...,...,...,...,...,...
3628,"[""morena""]",3240,Pendiente,propone establecer la prohibición de publicita...,"Comunicaciones y Transportes , Estudios Legisl...",Ley de Aviación Civil,ciudad de mexico,sesión ordinaria a distancia de la h. cámara ...,None,proyecto de decreto por el que se adiciona un ...
3629,"[""morena""]",7844,Pendiente,reducir las diputadas y los diputados por repr...,Puntos Constitucionales,constitución política de los estados unidos me...,ciudad de mexico,versión estenográfica de la sesión ordinaria d...,None,proyecto de decreto que reforma los artículos ...
3630,"[""morena""]",3721,Pendiente,propone establecer como objetivo de la polític...,"Desarrollo y Bienestar Social , Estudios Legis...",Ley de los Derechos de las Personas Adultas Ma...,ciudad de mexico,sesión ordinaria a distancia de la h. cámara ...,None,proyecto de decreto por el que se reforman y a...
3631,"[""morena""]",8876,Pendiente,reconocer dentro del sistema jurídico mexicano...,Desarrollo Social,ley general de desarrollo social,ciudad de mexico,versión estenográfica de la sesión ordinaria d...,None,proyecto de decreto que reforma los artículos ...


In [232]:
alianza_morena

,partidos,id,status,abstract,turno,laws_mod,status_date,state_pres,sess_pres,title
0,"[""prd"",""pvem""]",294,Pendiente,propone establecer la prohibición de la subcon...,"Trabajo y Previsión Social , Estudios Legislat...",Ley Federal de los Trabajadores al Servicio de...,ciudad de mexico,sesión ordinaria de la h. cámara de senadores...,None,proyecto de decreto por el que se adiciona un ...
1,"[""pri"",""morena""]",4553,Pendiente,celebrar audiencia dentro de los cinco días si...,Justicia Igualdad de Género,ley general de acceso de las mujeres a una vid...,ciudad de mexico,versión estenográfica de la sesión ordinaria d...,None,proyecto de decreto que reforma y adiciona los...
2,"[""prd"",""morena"",""pvem""]",460,Aprobada,la iniciativa propone reconocer en el texto co...,"Puntos Constitucionales , Estudios Legislativos",Constitución Política de los Estados Unidos Me...,"Tue, 30 Apr 2019 00:00:00 GMT",ciudad de mexico,sesión ordinaria de la h. cámara de senadores...,proyecto de decreto que reforma los artículos ...
3,"[""mc"",""morena""]",4354,Pendiente,precisar los derechos de los pueblos y comunid...,"Economía, Comercio y Competitividad Medio Ambi...",ley minera y de la ley general del equilibrio ...,ciudad de mexico,versión estenográfica de la sesión ordinaria d...,None,proyecto de decreto que adiciona y deroga dive...
4,"[""pri"",""morena""]",6544,Aprobada,aplicar una cuota de 15.00 pesos por kilogramo...,Hacienda y Crédito Público,ley del impuesto especial sobre producción y s...,"Thu, 17 Oct 2019 00:00:00 GMT",ciudad de mexico,versión estenográfica de la sesión ordinaria d...,proyecto de decreto que adiciona diversas disp...
...,...,...,...,...,...,...,...,...,...,...
951,"[""pt"",""pri"",""morena""]",323,Pendiente,"la iniciativa propone que el juez de distrito,...","Justicia , Estudios Legislativos Segunda","Ley de Amparo, Reglamentaria de los Artículos ...",ciudad de mexico,sesión ordinaria de la h. cámara de senadores...,None,proyecto de decreto por el que se reforma el a...
952,"[""pri"",""morena""]",5369,Pendiente,crear un ordenamiento jurídico reglamentario d...,Justicia Presupuesto y Cuenta Pública,"ley nacional de extinción de dominio, reglamen...",ciudad de mexico,versión estenográfica de la sesión ordinaria d...,None,proyecto de decreto que expide la ley nacional...
953,"[""pri"",""morena"",""pvem"",""mc""]",4327,Pendiente,constituir el fondo general de participaciones...,Hacienda y Crédito Público Federalismo y Desar...,"ley de coordinación fiscal, en materia de part...",ciudad de mexico,versión estenográfica de la sesión ordinaria d...,None,proyecto de decreto que reforma el artículo 2°...
954,"[""morena"",""pri""]",207,Pendiente,la iniciativa propone establecer mayores penas...,"Justicia , Estudios Legislativos Primera",Ley Federal de Responsabilidades Administrativ...,ciudad de mexico,sesión ordinaria de la h. cámara de senadores...,None,proyecto de decreto por el que se reforman los...


In [233]:
#títulos por si se quiere hacer análisis usando los títulos.

titulos = va_x_mex['title']
titulos.head().values

titulos_morena = alianza_morena['title']
titulos_morena.head().values

array(['proyecto de decreto por el que se adiciona un segundo párrafo al artículo 3º de la ley federal de los trabajadores al servicio del estado, reglamentaria del apartado b) del artículo 123 de la constitución política de los estados unidos mexicanos.',
       'proyecto de decreto que reforma y adiciona los artículos 137 del código nacional de procedimientos penales y 28 de la ley general de acceso de las mujeres a una vida libre de violencia.',
       'proyecto de decreto que reforma los artículos 2, 27, 28 y 115 de la constitución política de los estados unidos mexicanos, a efecto de reconocer al pueblo y comunidades afroamericanos, sus aportes a la cultura y a la historia de nuestro país, así como su participación en la conformación de la identidad nacional.',
       'proyecto de decreto que adiciona y deroga diversas disposiciones de la ley minera y de la ley general del equilibrio ecológico y la protección al ambiente.',
       'proyecto de decreto que adiciona diversas disposi

In [234]:
#para el análisis se van a usar los abstracts

abstracts = va_x_mex['abstract']
abstracts_morena = alianza_morena['abstract']

In [235]:
stop_words_sp = get_stop_words('spanish')

Para el anáñisis se va a realizar: 
- DTM (document term matrix)
- document frequency de cada término
- IDF para cada término de la DTM
- Calcular "relevancia" de los documentos

# Para las alianzas

In [236]:
tf_idf = TfidfVectorizer(lowercase=True, analyzer="word", stop_words=stop_words_sp, use_idf=True)
cv = CountVectorizer(lowercase=True, stop_words=stop_words_sp)


In [237]:
#dtm de va por méxico
dtm_q1 = cv.fit_transform(abstracts)
dtm_df = pd.DataFrame(dtm_q1.todense(), columns=cv.get_feature_names())

dtm_df

#dtm de juntos haremos historia
dtm_morena = cv.fit_transform(abstracts_morena)
dtm_morena_df = pd.DataFrame(dtm_morena.todense(), columns=cv.get_feature_names())

/home/ernesto_anaya/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [238]:
#term frequency

#para va por méxico
tf_arr = dtm_df.apply(lambda x: x.sum(), axis=0)
freq_20 = tf_arr.sort_values(ascending = False).head(20)

#para juntos haremos historia
tf_morena_arr = dtm_morena_df.apply(lambda x: x.sum(), axis=0)
freq_20_morena = tf_morena_arr.sort_values(ascending = False).head(20)

In [239]:
#para va por méxico
tokens_va_x_mex = pd.DataFrame()
tokens_va_x_mex['word'] = freq_20.index
tokens_va_x_mex['frequency'] = freq_20.values



tokens_va_x_mex

,word,frequency
0,propone,1194
1,establecer,1004
2,nacional,666
3,así,473
4,ley,467
5,personas,449
6,materia,421
7,derecho,381
8,derechos,358
9,federal,342


In [240]:
#para juntos haremos historia
tokens_morena = pd.DataFrame()
tokens_morena['word'] = freq_20_morena.index
tokens_morena['frequency'] = freq_20_morena.values

tokens_morena

,word,frequency
0,propone,298
1,establecer,241
2,nacional,223
3,ley,178
4,personas,168
5,derechos,152
6,crear,136
7,garantizar,131
8,así,129
9,derecho,125


In [258]:
#tf_idf

#de va por méxico
tfidf_m = tf_idf.fit(abstracts)
dtm = tfidf_m.transform(abstracts)

tf_idf_df = pd.DataFrame(dtm.todense(), columns=tf_idf.get_feature_names())

#de juntos haremos historia
tfidf_m_morena = tf_idf.fit(abstracts_morena)
dtm_morena = tfidf_m.transform(abstracts_morena)

tf_idf_df_morena = pd.DataFrame(dtm_morena.todense(), columns=tf_idf.get_feature_names())

/home/ernesto_anaya/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [262]:
#la suma de el TF-IDF de cada documento. Está ordenado de mayor a menor

#de va por méxico
tf_idf_arr = tf_idf_df.apply(lambda x: x.sum(), axis=0)
top_relevant = tf_idf_arr.sort_values(ascending = False).head(20)

#de juntos haremos historia
tf_idf_arr_morena = tf_idf_df_morena.apply(lambda x: x.sum(), axis=0)
top_relevant_morena = tf_idf_arr_morena.sort_values(ascending = False).head(20)

In [263]:
#abstracts[top_relevant.index]
top_relevant

establecer    83.687574
propone       81.498575
nacional      66.954801
derecho       49.010503
materia       48.495762
personas      47.753635
incluir       47.659520
ley           46.977942
género        45.074578
garantizar    43.208555
derechos      42.876345
así           41.528802
desarrollo    39.083140
federal       38.812754
salud         38.274609
educación     37.333576
social        36.406492
crear         36.086315
servicios     35.104195
objeto        34.397654
dtype: float64

In [264]:
top_relevant_morena

establecer      23.131697
propone         21.374068
nacional        19.658976
crear           16.642276
garantizar      15.544611
personas        15.287982
ley             14.980858
derecho         14.536214
derechos        14.483202
género          13.672693
objeto          13.511495
materia         13.449189
social          12.522729
desarrollo      12.174868
jurídico        12.053974
incluir         12.043661
ordenamiento    11.864096
educación       11.726042
violencia       11.704987
federal         11.307951
dtype: float64

In [273]:
#están por si se quiere ver cuantas veces se repiten las palabras en los abstracts. 

tf_idf = TfidfVectorizer(lowercase=True, analyzer="word", stop_words=stop_words_sp, use_idf=True)
tfidf_m = tf_idf.fit(abstracts)
dtm = tfidf_m.transform(abstracts)

tfidf_m = tfidf_m.vocabulary_

tfidf_m_morena = tf_idf.fit(abstracts_morena)
dtm_morena = tfidf_m_morena.transform(abstracts_morena)

tfidf_m_morena = tfidf_m_morena.vocabulary_

In [274]:
columns = {}

def get_relevance(query, top=5):
    q_tokens =  cv.fit(query)
    tokens = q_tokens.get_feature_names_out()
    print("Tokens del query:\n", tokens)
    
    for element in tokens:
        columns[element] = tfidf_m[element]
        
    # filtramos solo los términos que existen en nuestra DTM
    dtm_q = dtm[:,list(columns.values())].todense()
    # convertimos a DF para que la manipulación sea más sencilla
    tdm_df = pd.DataFrame(dtm_q)
    # sumamos td-idf de todos los documentos en la TDM para los tokens en el query
    tdidf_by_doc = tdm_df.apply(lambda x: x.sum(), axis=1)
    # top n de relevancia, por default 5
    top_n_relevance = tdidf_by_doc.sort_values(ascending=False).head(top)
    
    return (abstracts[top_n_relevance.index], top_n_relevance)

In [275]:
#en get_relevance hay que auitar los stopwords del query

query = abstracts.mode()[0]
query = [remove_stopwords(query)]
np.asarray(get_relevance(query,20))


Tokens del query:
 ['abrogar' 'agua' 'aguas' 'comunidades' 'comunitario' 'expedir' 'fin'
 'fortalecer' 'general' 'gestión' 'ley' 'locales' 'manejo' 'mayor'
 'nacionales' 'participación' 'permitir' 'reconociendo' 'social']


array([['expedir la ley general de aguas y abrogar la ley de aguas nacionales, con el fin de fortalecer la gestión del agua, y permitir una mayor participación social de comunidades locales, reconociendo el manejo comunitario del agua.',
        'expedir la ley general de aguas y abrogar la ley de aguas nacionales, con el fin de fortalecer la gestión del agua, y permitir una mayor participación social de comunidades locales, reconociendo el manejo comunitario del agua.',
        'expedir la ley general de aguas y abrogar la ley de aguas nacionales, con el fin de fortalecer la gestión del agua, y permitir una mayor participación social de comunidades locales, reconociendo el manejo comunitario del agua.',
        'expedir la ley general de aguas y abrogar la ley de aguas nacionales, con el fin de fortalecer la gestión del agua, y permitir una mayor participación social de comunidades locales, reconociendo el manejo comunitario del agua.',
        'expedir la ley general de aguas y abrog

In [286]:
query = abstracts_morena.mode()[0]
query = [remove_stopwords(query)]
np.asarray(get_relevance(query,20))

Tokens del query:
 ['abrogar' 'administración' 'agua' 'aguas' 'apoyos' 'aprovechamiento'
 'asociados' 'bases' 'crear' 'definir' 'derechos' 'disposición'
 'ecosistemas' 'garantizar' 'humanos' 'inherentes' 'jurídico' 'ley'
 'modalidades' 'nacionales' 'objeto' 'ordenamiento' 'principios'
 'promover' 'proteger' 'respetar' 'saneamiento' 'sustentabilidad']


array([['expedir la ley general de aguas y abrogar la ley de aguas nacionales, con el fin de fortalecer la gestión del agua, y permitir una mayor participación social de comunidades locales, reconociendo el manejo comunitario del agua.',
        'expedir la ley general de aguas y abrogar la ley de aguas nacionales, con el fin de fortalecer la gestión del agua, y permitir una mayor participación social de comunidades locales, reconociendo el manejo comunitario del agua.',
        'expedir la ley general de aguas y abrogar la ley de aguas nacionales, con el fin de fortalecer la gestión del agua, y permitir una mayor participación social de comunidades locales, reconociendo el manejo comunitario del agua.',
        'expedir la ley general de aguas y abrogar la ley de aguas nacionales, con el fin de fortalecer la gestión del agua, y permitir una mayor participación social de comunidades locales, reconociendo el manejo comunitario del agua.',
        'expedir la ley general de aguas y abrog

In [287]:
#

q1 = ["tipificar delitos materia de hostigamiento y acoso sexual"]

cv = CountVectorizer(lowercase=True, stop_words='english')
q1_tokens = cv.fit(q1)

#q1_tokens.get_feature_names() ya está deprecado
q1_tokens.get_feature_names_out()

array(['acoso', 'delitos', 'hostigamiento', 'materia', 'sexual',
       'tipificar'], dtype=object)

In [288]:
columns = {}

for element in q1_tokens.get_feature_names():
    #print(terms[element]
    columns[element] = tfidf_m[element]

# Para covid

In [249]:
tf_idf_df

,00,000,01,025,03,09,10,100,1000,101,...,última,últimas,último,últimos,única,únicamente,único,únicos,útil,útiles
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3629,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3631,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [250]:
dtm_df

,00,000,01,025,03,09,10,100,1000,101,...,última,últimas,último,últimos,única,únicamente,único,únicos,útil,útiles
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3628,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3629,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3630,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3631,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [253]:
tokens_va_x_mex

,word,frequency
0,propone,1194
1,establecer,1004
2,nacional,666
3,así,473
4,ley,467
5,personas,449
6,materia,421
7,derecho,381
8,derechos,358
9,federal,342


In [254]:
tokens_morena

,word,frequency
0,propone,298
1,establecer,241
2,nacional,223
3,ley,178
4,personas,168
5,derechos,152
6,crear,136
7,garantizar,131
8,así,129
9,derecho,125
